<a href="https://colab.research.google.com/github/amitchug/ALMlops/blob/main/Hf_Fine_Tuning_BERT_using_Built_in_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Huggingface: Fine Tunning BERT using Built in Data**

### Objectives:

At the end of the experiment you will be able to understand  :

* How datasets are represented in HF
* Trainer and trainingArguments objects
* Computing Metrics
* Saving and loading the trained model

### Install dependencies

After installing the below dependencies ***Restart the session/runtime***.

Accelerate is a library that enables the same PyTorch code to be run across any distributed configuration by adding just four lines of code, making training and inference at scale made simple, efficient and adaptable.

In [ ]:
!pip -q uninstall pyarrow -y
!pip -q install pyarrow==15.0.2
!pip -q install datasets              # Hugging Face Datasets Library
!pip -q install accelerate
!pip -q install transformers

# Ignore the Error/Warning showing after running this cell

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 19.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 15.0.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00


### <font color="#990000">Restart Session/Runtime</font>

### **Loading the dataset**
Built in data sets are included with the data sets package. Using a single line of code, we can load in the data set  which  are well formatted  and processed for us to use.

In [ ]:
import numpy as np
from datasets import load_dataset
#rd1=load_dataset("amazon_polarity") # dataset for sentiment analysis
rd2=load_dataset("glue","sst2")     # Part of GLUE Benchmark

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Calling the load data set function, returns a data set dict object, which is like a dictionary that may contain multiple data sets like -  a train dataset, validation dataset and test dataset.

In [ ]:
rd2

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

Getting different data and individual entries

In [ ]:
rd2['train']

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 67349
})

In [ ]:
print(rd2['train'], '\n')
type(rd2['train'])

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 67349
}) 



datasets.arrow_dataset.Dataset

In [ ]:
rd2['train'].data

MemoryMappedTable
sentence: string
label: int64
idx: int32
----
sentence: [["hide new secretions from the parental units ","contains no wit , only labored gags ","that loves its characters and communicates something rather beautiful about human nature ","remains utterly satisfied to remain the same throughout ","on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ",...,"you wish you were at home watching that movie instead of in the theater watching this one ","'s no point in extracting the bare bones of byatt 's plot for purposes of bland hollywood romance ","underdeveloped ","the jokes are flat ","a heartening tale of small victories "],["suspense , intriguing characters and bizarre bank robberies , ","a gritty police thriller with all the dysfunctional family dynamics one could wish for ","with a wonderful ensemble cast of characters that bring the routine day to day struggles of the working class to life ","nonetheless appreciates the art and reveals a music sc

In [ ]:
rd2['train'][0]

{'sentence': 'hide new secretions from the parental units ',
 'label': 0,
 'idx': 0}

In [ ]:
rd2['train'][10:13]

{'sentence': ['goes to absurd lengths ',
  "for those moviegoers who complain that ` they do n't make movies like they used to anymore ",
  "the part where nothing 's happening , "],
 'label': [0, 0, 0],
 'idx': [10, 11, 12]}

In [ ]:
rd2['train'].features

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None),
 'idx': Value(dtype='int32', id=None)}

### **Tokenization with DataSets**
Recall that previously, we mentioned the need for truncation, padding and converting the tokenized data into PyTorch Tensors. Here, all of these steps will be handled  behind the scenes and no need to do anything. We are going to do is to partially apply the tokenizer to the data set dict object.



In [ ]:
from transformers import AutoTokenizer

In [ ]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
test_sent = rd2['train']['sentence'][0]
test_sent

'hide new secretions from the parental units '

In [ ]:
tokenized_sent = tokenizer(test_sent)
tokenized_sent

{'input_ids': [101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

IDs 101 and 102 are special tokens that indicate the beginning and end of an input sequence, respectively. These are represented by the string values [CLS] and [SEP], and are inserted automatically into the tokenizer output.

In [ ]:
decoded_sent = tokenizer.decode(tokenized_sent['input_ids'])
decoded_sent

'[CLS] hide new secretions from the parental units [SEP]'

In [ ]:
tokenized_sentence = tokenizer(rd2['train'][0:3]['sentence'])
from pprint import pprint
pprint(tokenized_sentence)

{'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
 'input_ids': [[101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102],
               [101,
                3397,
                2053,
                15966,
                1010,
                2069,
                4450,
                2098,
                18201,
                2015,
                102],
               [101,
                2008,
                7459,
                2049,
                3494,
                1998,
                10639,
                2015,
                2242,
                2738,
                3376,
                2055,
                2529,
                3267,
                102]]}


By calling the map method and passing in a tokenized function, the data set library automatically knows it should apply the same function to every data set i.e.train, validation and test.

 Note that in this example we apply only truncation but not padding or conversion into Pytorch Tensors. This will be handled behind the scenes by the trainer object we create later.

In [ ]:
def tokenize_fn(batch):
  return tokenizer(batch['sentence'],truncation=True)

In [ ]:
tokenized_datasets=rd2.map(tokenize_fn,batched=True)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

### **Loading Pre-Trained Model**

In [ ]:
from transformers import AutoModelForSequenceClassification
model =AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
type(model)

transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification

In [ ]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
!pip install torchinfo

In [ ]:
from torchinfo import summary
summary(model)

Layer (type:depth-idx)                                  Param #
DistilBertForSequenceClassification                     --
├─DistilBertModel: 1-1                                  --
│    └─Embeddings: 2-1                                  --
│    │    └─Embedding: 3-1                              23,440,896
│    │    └─Embedding: 3-2                              393,216
│    │    └─LayerNorm: 3-3                              1,536
│    │    └─Dropout: 3-4                                --
│    └─Transformer: 2-2                                 --
│    │    └─ModuleList: 3-5                             42,527,232
├─Linear: 1-2                                           590,592
├─Linear: 1-3                                           1,538
├─Dropout: 1-4                                          --
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0

#### **Adding custom classifier layer and Freezing/Un-Freezing different layers**

In [ ]:
for name, param in model.named_parameters():
  print(name)

distilbert.embeddings.word_embeddings.weight
distilbert.embeddings.position_embeddings.weight
distilbert.embeddings.LayerNorm.weight
distilbert.embeddings.LayerNorm.bias
distilbert.transformer.layer.0.attention.q_lin.weight
distilbert.transformer.layer.0.attention.q_lin.bias
distilbert.transformer.layer.0.attention.k_lin.weight
distilbert.transformer.layer.0.attention.k_lin.bias
distilbert.transformer.layer.0.attention.v_lin.weight
distilbert.transformer.layer.0.attention.v_lin.bias
distilbert.transformer.layer.0.attention.out_lin.weight
distilbert.transformer.layer.0.attention.out_lin.bias
distilbert.transformer.layer.0.sa_layer_norm.weight
distilbert.transformer.layer.0.sa_layer_norm.bias
distilbert.transformer.layer.0.ffn.lin1.weight
distilbert.transformer.layer.0.ffn.lin1.bias
distilbert.transformer.layer.0.ffn.lin2.weight
distilbert.transformer.layer.0.ffn.lin2.bias
distilbert.transformer.layer.0.output_layer_norm.weight
distilbert.transformer.layer.0.output_layer_norm.bias
distil

In [ ]:
# Freezing
for name, param in model.named_parameters():
     if name.startswith("distilbert"): # choose whatever you like here
        param.requires_grad = False

In [ ]:
# Displaying of layers gradient
for name, param in model.named_parameters():
     print(name, param.requires_grad)

distilbert.embeddings.word_embeddings.weight False
distilbert.embeddings.position_embeddings.weight False
distilbert.embeddings.LayerNorm.weight False
distilbert.embeddings.LayerNorm.bias False
distilbert.transformer.layer.0.attention.q_lin.weight False
distilbert.transformer.layer.0.attention.q_lin.bias False
distilbert.transformer.layer.0.attention.k_lin.weight False
distilbert.transformer.layer.0.attention.k_lin.bias False
distilbert.transformer.layer.0.attention.v_lin.weight False
distilbert.transformer.layer.0.attention.v_lin.bias False
distilbert.transformer.layer.0.attention.out_lin.weight False
distilbert.transformer.layer.0.attention.out_lin.bias False
distilbert.transformer.layer.0.sa_layer_norm.weight False
distilbert.transformer.layer.0.sa_layer_norm.bias False
distilbert.transformer.layer.0.ffn.lin1.weight False
distilbert.transformer.layer.0.ffn.lin1.bias False
distilbert.transformer.layer.0.ffn.lin2.weight False
distilbert.transformer.layer.0.ffn.lin2.bias False
distilbe

#### **Modifying last classifier layers**

In [ ]:
#import torch

In [ ]:
#model.pre_classifier

Linear(in_features=768, out_features=768, bias=True)

In [ ]:
#print(model.pre_classifier)
#model.pre_classifier = torch.nn.Linear(768,500)
#print(model.pre_classifier)

Linear(in_features=768, out_features=768, bias=True)
Linear(in_features=768, out_features=500, bias=True)


In [ ]:
##print(model.classifier)
#model.classifier = torch.nn.Linear(500,2)
#print(model.classifier)

Linear(in_features=768, out_features=2, bias=True)
Linear(in_features=500, out_features=2, bias=True)


In [ ]:
#model

### **Metrics**

In HuggingFace, there are predefined metrics for the built in data sets and we are going to use that but we can define custom metrics as well.

Specifically, there is a function called the load metric, also part of the data sets library. The arguments to this happens to be the same as the arguments into load data set for this example.

More generally, however, you can pass in an argument to specify a specific metric like the Blue score or a python file where you've defined your own computations.

In [ ]:
from datasets import load_metric
metric =load_metric("glue","sst2")

<ipython-input-23-9b067145172b>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric =load_metric("glue","sst2")


The repository for glue contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/glue.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


But,  we cannot pass this metric into the trainer directly. we need to define another function to do some extra work.

Firstly, the metric we get back from `load_metric` is an object which has a method called `compute`, which will return a dictionary containing the computed metrics for a given task. This compute method it takes in to erase the predictions and the targets called a references.

This will give us back a dictionary with the value of the metric or metrics with their names as keys.

In [ ]:
metric.compute(predictions=[1,0,1], references=[1,0,0])

{'accuracy': 0.6666666666666666}

In [ ]:
def compute_metrics(logits_and_labels):
  logits, labels = logits_and_labels
  predictions =np.argmax(logits,axis=-1)
  return metric.compute(predictions=predictions,references=labels)

### **Training argument class**


The training arguments class is like a training configuration. It allows to specify things like  - `where to save the outputs of the training process, how often to compute metrics, how many epochs to train for the learning rate, and many more`. There are tons of arguments into the constructor for this class. So check out the documentation.

Note that the HuggingFace trainer uses an optimizer called **`Adam W`** by default, which is a slight variation of Adam.This is essentially built in and it's good enough for our purposes.If you want to use a different optimizer, you can always write your own custom training loop in plain Pytorch syntax as well.

In [ ]:
from transformers import TrainingArguments

In [ ]:
training_args = TrainingArguments(
    'sample_trainer', #  directory where the trained model will be saved,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=1

)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### **Trainer**

Trainer object is what we use to run the training process. The arguments to the constructor are quite simple. We pass in the model, the trainer arguments, the training data set, the validation data set, the tokenizer and the metrics.

Now, just call `trainer` and apply `train` method to train.



In [ ]:
from transformers import Trainer

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    tokenizer =tokenizer,
    compute_metrics=compute_metrics

)

In [ ]:
trainer.train()
 # ~ 2 mint for tunning just last classifier layers
 # ~ 8 mint for tunning whole architecture

Epoch,Training Loss,Validation Loss,Accuracy
1,0.367300,0.386842,0.830275


TrainOutput(global_step=8419, training_loss=0.39978386203499894, metrics={'train_runtime': 109.5273, 'train_samples_per_second': 614.906, 'train_steps_per_second': 76.867, 'total_flos': 517212489917652.0, 'train_loss': 0.39978386203499894, 'epoch': 1.0})

#### **Saving the model**

In [ ]:
trainer.save_model('saved_model')

In [ ]:
!ls

sample_data  sample_trainer  saved_model


#### **Loading the saved model**

In [ ]:
from transformers import pipeline

In [ ]:
my_model = pipeline(
    'text-classification',
    model='saved_model',
    device=0)

In [ ]:
my_model('this is a very good book to read')

[{'label': 'LABEL_1', 'score': 0.9971581697463989}]